In [ ]:
#Inception Module


# Load pickled data
import pickle
import numpy as np
import tensorflow as tf 

# TODO: Fill this in based on where you saved the training and testing data

training_file = "train_gray_balanced_contrast.p"
testing_file = "test_final_gray.p"
validation_file = "validation_gray_balanced.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
# with open(testing_file, mode='rb') as f:
#     test = pickle.load(f)
with open(validation_file, mode='rb') as f:
    validation = pickle.load(f)
    
X_train_old, y_train = train['features'], train['labels']
# X_test, y_test = test['features'], test['labels']
X_validation_old, y_validation = validation['features'], validation['labels']


In [ ]:
from sklearn.preprocessing import normalize

X_new = X_train_old#.astype(float)
X_new = X_new.astype(float)
X_new-=128.0
X_train = X_new /255.0

X_val = X_validation_old#.astype(float)
X_val = X_val.astype(float)
X_val-=128.0
X_validation = X_val /255.0

# X_test = X_test#.astype(float)
# X_test = X_test.astype(float)
# X_test-=128.0
# X_test = X_test /255.0

In [ ]:

from tensorflow.contrib.layers import flatten
import tensorflow as tf


def conv2d(x,weight,bias,strides=1,padding='SAME'):
    layer = tf.nn.conv2d(x,weight,[1,strides,strides,1],padding=padding)
    layer = tf.nn.bias_add(layer,bias)
    layer = tf.nn.relu(layer)
    return layer

def maxpool(x,k=2,padding='SAME',stride=2):
    kernel_size = [1,k,k,1]
    return tf.nn.max_pool(x,ksize=kernel_size,strides=[1,stride,stride,1],padding=padding)
    

In [ ]:

#https://hacktilldawn.com/2016/09/25/inception-modules-explained-and-implemented/
def naive_inception(x,d):
    
    mu = 0
    sigma = 0.1
    in_depth = x.get_shape().as_list()[3]
    layer_depth = {
    'layer_1_1x1': d,
    'layer_1_3x3': d,   
    'layer_1_5x5': d
    }
    
    #1x1 conv
    weights_1x1 = tf.Variable(tf.random_normal([1, 1,in_depth , layer_depth['layer_1_1x1']], mean=0.0, stddev=1.0, dtype=tf.float32) )
    bias_1x1 = tf.Variable(tf.zeros(layer_depth['layer_1_1x1']))
    
    conv_1x1 = conv2d(x,weights_1x1,bias_1x1)
    print(conv_1x1.get_shape())
    
    #3x3 conv
    weights_3x3 = tf.Variable(tf.random_normal([3, 3,in_depth , layer_depth['layer_1_3x3']], mean=0.0, stddev=1.0, dtype=tf.float32) )
    bias_3x3 = tf.Variable(tf.zeros(layer_depth['layer_1_3x3']))
    
    conv_3x3 = conv2d(x,weights_3x3,bias_3x3)
    print(conv_3x3.get_shape())

    #5x5 conv
    weights_5x5 = tf.Variable(tf.random_normal([5, 5,in_depth , layer_depth['layer_1_5x5']], mean=0.0, stddev=1.0, dtype=tf.float32) )
    bias_5x5 = tf.Variable(tf.zeros(layer_depth['layer_1_5x5']))
    
    conv_5x5 = conv2d(x,weights_5x5,bias_5x5)
    print(conv_5x5.get_shape())

    #3x3 maxpooling
    
    maxpool_3x3 = maxpool(x,k=3,stride=1)
    print(maxpool_3x3.get_shape())

    #stack 
    incepction_tensor = tf.concat(3,(conv_1x1,conv_3x3,conv_5x5,maxpool_3x3))
    
    return incepction_tensor

def fully_connected(in_tensor,layers,keep_prob):
    mu = 0
    sigma = 0.1
    in_dim = np.product(in_tensor.get_shape().as_list()[1:])
    tensor_flatten = flatten(in_tensor)
    
    for each in layers:
        weights = tf.Variable(tf.random_normal([in_dim, each],mu,sigma,dtype=tf.float32))
        bias =  tf.Variable(tf.zeros(each))

        fc1_drop = tf.nn.dropout(tensor_flatten, keep_prob)    
        fc2 = tf.add(
        tf.matmul(fc1_drop, weights),
        bias)
        # TODO: Activation.
        tensor_flatten = tf.nn.tanh(fc2)
        in_dim = each
        print(in_dim)
    
    return tensor_flatten
         
        
    


In [ ]:
INPUT_DEPTH = 1
import tensorflow.contrib.slim as slim

tf.reset_default_graph()
x = tf.placeholder(tf.float32, (None, 32, 32, INPUT_DEPTH))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)
#  INCEPTION 1
incep_tensor = naive_inception(x,16)
batch_norm_incep_1 = slim.batch_norm(incep_tensor,activation_fn=None)
input_layer_2 = tf.nn.relu(batch_norm_incep_1)
layer_2 = maxpool(input_layer_2,k=2)
layer_2 = tf.nn.dropout(layer_2, keep_prob)

#INCEPTION 2
inception_2 = naive_inception(layer_2,22)
batch_norm_incep_2 = slim.batch_norm(inception_2,activation_fn=None)
input_layer_3 = tf.nn.relu(batch_norm_incep_2)
layer_3 = maxpool(input_layer_3,k=2)
layer_3 = tf.nn.dropout(layer_3, keep_prob)

#INCEPTION 3
inception_3 = naive_inception(layer_3,42)
batch_norm_incep_3 = slim.batch_norm(inception_3,activation_fn=None)
input_layer_4 = tf.nn.relu(batch_norm_incep_3)
layer_4 = maxpool(input_layer_4,k=2)
layer_4 = tf.nn.dropout(layer_4, keep_prob)

logits = fully_connected(layer_4,[512,512,43],keep_prob)

In [ ]:
rate = 0.002
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
# Gradient clipping
gvs = optimizer.compute_gradients(loss_operation)
capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
optimizer.apply_gradients(capped_gvs)

training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
EPOCHS = 100
BATCH_SIZE = 128

In [ ]:
from sklearn.utils import shuffle
from imblearn.over_sampling import RandomOverSampler
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
model_file = "./inception_naive_gray_contrast_v1"
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples =len(X_train)#392090
    from pathlib import Path
    

    my_file = Path(model_file+".meta")
    if my_file.is_file():
        saver.restore(sess,model_file)
        print("Model restored...")
    print("Training...")
    print()
    ros = RandomOverSampler()
#     x_index = np.arange(0,len(y_train),1)
#     x_index_resampled, y_resampled = ros.fit_sample(np.expand_dims(x_index,axis=1), y_train)
#     x_in = np.squeeze(x_index_resampled)
#     X_resampled = X_train[x_in,:,:,]
#     X_train_1, y_train_1 = shuffle(X_resampled, y_resampled)
    X_train_1 = X_train
    y_train_1 = y_train
    for i in range(EPOCHS):

        X_train, y_train = shuffle(X_train_1, y_train_1)
        acc = []
        ent = []
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            
            batch_x, batch_y = np.copy(X_train[offset:end]), np.copy(y_train[offset:end])
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1.0})
            training_accuracy = evaluate(batch_x, batch_y)
            mean_cross_entropy = sess.run(loss_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1.0})
            acc.append(training_accuracy)
            ent.append(mean_cross_entropy)
            
        validation_accuracy = evaluate(X_validation, y_validation)
        
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(np.mean(acc)))
        print("Cross entropy = {:.4f}".format(np.mean(ent)))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        
        if(i%10==0):
            saver.save(sess, model_file)
            print("Model saved")
        
    saver.save(sess, model_file)
    print("Model saved")

In [ ]:
def LeNet_multi(x,keep_prob):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    layer_depth = {
    'layer_1': 9,
    'layer_2': 16,
    'fully_connected_1': 120,
#     'fully_connected_2': 84,
    'fully_connected_3': 43
    }
    
    weights = {
        'layer_1': tf.Variable(tf.random_normal([5, 5, 1, layer_depth['layer_1']], mean=0.0, stddev=1.0, dtype=tf.float32) ),
        'layer_2': tf.Variable(tf.random_normal(
                [5, 5, layer_depth['layer_1'], layer_depth['layer_2']],mu,sigma,dtype=tf.float32)),
        'fully_connected_1':tf.Variable(tf.random_normal(
                [5*5*16+14*14*9, layer_depth['fully_connected_1']],mu,sigma,dtype=tf.float32)),
#         'fully_connected_2':tf.Variable(tf.random_normal(
#                [layer_depth['fully_connected_1'], layer_depth['fully_connected_2']],mu,sigma,dtype=tf.float32)),
        'fully_connected_3':tf.Variable(tf.random_normal(
                [layer_depth['fully_connected_1'], layer_depth['fully_connected_3']],mu,sigma,dtype=tf.float32))
    }
    biases = {
        'layer_1': tf.Variable(tf.zeros(
                layer_depth['layer_1'])),
        'layer_2': tf.Variable(tf.zeros(
                layer_depth['layer_2'])),
        'fully_connected_1':tf.Variable(tf.zeros( 
                layer_depth['fully_connected_1'])),
#         'fully_connected_2':tf.Variable(tf.zeros( 
#                layer_depth['fully_connected_2'])),
        'fully_connected_3':tf.Variable(tf.zeros( 
                layer_depth['fully_connected_3'])),
    }
    
    # TODO: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    # TODO: Activation.
    layer_1 = conv2d(x,weights['layer_1'],biases['layer_1'],strides=1,padding='VALID')
    print(layer_1.get_shape())
    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    layer_1 = maxpool(layer_1,k=2)
    print(layer_1.get_shape())
    
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    
    # TODO: Activation.
    layer_2 = conv2d(layer_1,weights['layer_2'],biases['layer_2'],strides=1,padding='VALID')
    print(layer_2.get_shape())
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    layer_2 = maxpool(layer_2,k=2)
    print(layer_2.get_shape())
    
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    
    fc1 = flatten(layer_2)
    fc1 = tf.concat(1,(fc1,flatten(layer_1)))
    
    
    fc1_drop = tf.nn.dropout(fc1, keep_prob)

    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    
    fc1 = tf.add(
        tf.matmul(fc1_drop, weights['fully_connected_1']),
        biases['fully_connected_1'])
    # TODO: Activation.
    fc1= tf.nn.tanh(fc1)
    
    
    fc1_drop = tf.nn.dropout(fc1, keep_prob)
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    
#     fc2 = tf.add(
#         tf.matmul(fc1_drop, weights['fully_connected_2']),
#         biases['fully_connected_2'])
#     # TODO: Activation.
#     fc2= tf.nn.tanh(fc2)
    fc2_drop = fc1_drop#tf.nn.dropout(fc2, keep_prob)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3 = tf.add(
        tf.matmul(fc2_drop, weights['fully_connected_3']),
        biases['fully_connected_3'])
    # TODO: Activation.
    logits= tf.nn.tanh(fc3)
    return logits

In [ ]:
   
def LeNet(x,keep_prob):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    layer_depth = {
    'layer_1': 6,
    'layer_2': 16,
    'fully_connected_1': 120,
    'fully_connected_2': 84,
    'fully_connected_3': 43
    }
    
    weights = {
        'layer_1': tf.Variable(tf.random_normal([5, 5, 1, layer_depth['layer_1']], mean=0.0, stddev=1.0, dtype=tf.float32) ),
        'layer_2': tf.Variable(tf.random_normal(
                [5, 5, layer_depth['layer_1'], layer_depth['layer_2']],mu,sigma,dtype=tf.float32)),
        'fully_connected_1':tf.Variable(tf.random_normal(
                [5*5*16, layer_depth['fully_connected_1']],mu,sigma,dtype=tf.float32)),
        'fully_connected_2':tf.Variable(tf.random_normal(
               [layer_depth['fully_connected_1'], layer_depth['fully_connected_2']],mu,sigma,dtype=tf.float32)),
        'fully_connected_3':tf.Variable(tf.random_normal(
                [layer_depth['fully_connected_2'], layer_depth['fully_connected_3']],mu,sigma,dtype=tf.float32))
    }
    biases = {
        'layer_1': tf.Variable(tf.zeros(
                layer_depth['layer_1'])),
        'layer_2': tf.Variable(tf.zeros(
                layer_depth['layer_2'])),
        'fully_connected_1':tf.Variable(tf.zeros( 
                layer_depth['fully_connected_1'])),
        'fully_connected_2':tf.Variable(tf.zeros( 
               layer_depth['fully_connected_2'])),
        'fully_connected_3':tf.Variable(tf.zeros( 
                layer_depth['fully_connected_3'])),
    }
    
    # TODO: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    # TODO: Activation.
    layer_1 = conv2d(x,weights['layer_1'],biases['layer_1'],strides=1,padding='VALID')
    print(layer_1.get_shape())
    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    layer_1 = maxpool(layer_1,k=2)
    print(layer_1.get_shape())
    
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    
    # TODO: Activation.
    layer_2 = conv2d(layer_1,weights['layer_2'],biases['layer_2'],strides=1,padding='VALID')
    print(layer_2.get_shape())
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    layer_2 = maxpool(layer_2,k=2)
    print(layer_2.get_shape())
    
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc1 = flatten(layer_2)
    
    
    fc1_drop = tf.nn.dropout(fc1, keep_prob)

    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    
    fc1 = tf.add(
        tf.matmul(fc1_drop, weights['fully_connected_1']),
        biases['fully_connected_1'])
    # TODO: Activation.
    fc1= tf.nn.tanh(fc1)
    
    
    fc1_drop = tf.nn.dropout(fc1, keep_prob)
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    
    fc2 = tf.add(
        tf.matmul(fc1_drop, weights['fully_connected_2']),
        biases['fully_connected_2'])
    # TODO: Activation.
    fc2= tf.nn.tanh(fc2)
    fc2_drop = tf.nn.dropout(fc2, keep_prob)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3 = tf.add(
        tf.matmul(fc2_drop, weights['fully_connected_3']),
        biases['fully_connected_3'])
    # TODO: Activation.
    logits= tf.nn.tanh(fc3)
    return logits

In [ ]:
def MyNet(x,keep_prob):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    layer_depth = {
    'layer_1': 9,
    'layer_2': 16,
    'fully_connected_1': 120,
    'fully_connected_2': 84,
    'fully_connected_3': 43
    }
    
    layer_depth_second = {
    'layer_1': 9,
    'layer_2': 16
        }
    weights_second = {
        'layer_1': tf.Variable(tf.random_normal([7, 7,1 , layer_depth_second['layer_1']], mean=0.0, stddev=1.0, dtype=tf.float32) ),
        'layer_2': tf.Variable(tf.random_normal(
                [7, 7, layer_depth_second['layer_1'], layer_depth_second['layer_2']],mu,sigma,dtype=tf.float32))
        }
    biases_second = {
        'layer_1': tf.Variable(tf.zeros(
                layer_depth_second['layer_1'])),
        'layer_2': tf.Variable(tf.zeros(
                layer_depth_second['layer_2']))
        }
    
    weights = {
        'layer_1': tf.Variable(tf.random_normal([5, 5,1 , layer_depth['layer_1']], mean=0.0, stddev=1.0, dtype=tf.float32) ),
        'layer_2': tf.Variable(tf.random_normal(
                [5, 5, layer_depth['layer_1'], layer_depth['layer_2']],mu,sigma,dtype=tf.float32)),
        'fully_connected_1':tf.Variable(tf.random_normal(
                [5*5*16+4*4*16, layer_depth['fully_connected_1']],mu,sigma,dtype=tf.float32)),
        'fully_connected_2':tf.Variable(tf.random_normal(
                [layer_depth['fully_connected_1'], layer_depth['fully_connected_2']],mu,sigma,dtype=tf.float32)),
        'fully_connected_3':tf.Variable(tf.random_normal(
                [layer_depth['fully_connected_2'], layer_depth['fully_connected_3']],mu,sigma,dtype=tf.float32))
    }
    biases = {
        'layer_1': tf.Variable(tf.zeros(
                layer_depth['layer_1'])),
        'layer_2': tf.Variable(tf.zeros(
                layer_depth['layer_2'])),
        'fully_connected_1':tf.Variable(tf.zeros( 
                layer_depth['fully_connected_1'])),
        'fully_connected_2':tf.Variable(tf.zeros( 
                layer_depth['fully_connected_2'])),
        'fully_connected_3':tf.Variable(tf.zeros( 
                layer_depth['fully_connected_3'])),
    }
    
    ####### SECOND #########
    second_layer_1 = conv2d(x,weights_second['layer_1'],biases_second['layer_1'],strides=1,padding='VALID')
    print(second_layer_1.get_shape())
    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    second_layer_1 = maxpool(second_layer_1,k=2)
    print(second_layer_1.get_shape())
    
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    
    # TODO: Activation.
    second_layer_2 = conv2d(second_layer_1,weights_second['layer_2'],biases_second['layer_2'],strides=1,padding='VALID')
    print(second_layer_2.get_shape())
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    second_layer_2 = maxpool(second_layer_2,k=2)
    print(second_layer_2.get_shape())
    
    fc1_second = flatten(second_layer_2)
    
    ###################
    
    
    # TODO: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    # TODO: Activation.
    layer_1 = conv2d(x,weights['layer_1'],biases['layer_1'],strides=1,padding='VALID')
    print(layer_1.get_shape())
    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    layer_1 = maxpool(layer_1,k=2)
    print(layer_1.get_shape())
    
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    
    # TODO: Activation.
    layer_2 = conv2d(layer_1,weights['layer_2'],biases['layer_2'],strides=1,padding='VALID')
    print(layer_2.get_shape())
    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    layer_2 = maxpool(layer_2,k=2)
    print(layer_2.get_shape())
    
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc1 = flatten(layer_2)
    fc1 = tf.concat(1,[fc1_second,fc1])#(,0))
    print(fc1.get_shape())
    
    
    fc1_drop = tf.nn.dropout(fc1, keep_prob)

    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    
    fc1 = tf.add(
        tf.matmul(fc1_drop, weights['fully_connected_1']),
        biases['fully_connected_1'])
    # TODO: Activation.
    fc1= tf.nn.tanh(fc1)
    
    
    fc1_drop = tf.nn.dropout(fc1, keep_prob)
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    
    fc2 = tf.add(
        tf.matmul(fc1_drop, weights['fully_connected_2']),
        biases['fully_connected_2'])
    # TODO: Activation.
    fc2= tf.nn.tanh(fc2)
    fc2_drop = tf.nn.dropout(fc2, keep_prob)

    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3 = tf.add(
        tf.matmul(fc2_drop, weights['fully_connected_3']),
        biases['fully_connected_3'])
    # TODO: Activation.
    logits= tf.nn.tanh(fc3)
    return logits